In [1]:
%load_ext lab_black

Step: Load data

In [2]:
import csv
import pandas as pd
import numpy as np
import time

import os


from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig, Dataset
from azureml.pipeline.core import Pipeline, ScheduleRecurrence, Schedule
from azureml.pipeline.steps import PythonScriptStep

from azureml.core import Workspace, Dataset

workspace = Workspace.from_config()
datastore = workspace.get_default_datastore()

In [6]:
# Set up the environment
environment = Environment(name="nfl-scraper")
environment.docker.enabled = True
environment.docker.base_image = None
environment.docker.base_dockerfile = "Dockerfile"

'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [17]:
project_directory = os.path.abspath(".")
project_directory

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/matthewhanauer991/code/Users/matthewhanauer99/draft_kings_analyses/src/production'

In [18]:
# Create a ScriptRunConfig
src = ScriptRunConfig(
    source_directory=project_directory,
    script="production_injury_data.py",
    environment=environment,
)

# Create a PythonScriptStep
step = PythonScriptStep(
    name="nfl-scraper",
    script_name="production_injury_data.py",
    source_directory=project_directory,
    compute_target="matthewhanauer991",
    runconfig=src.run_config,
)

# Create a Pipeline
pipeline = Pipeline(workspace=workspace, steps=[step])


# Create an Experiment
experiment = Experiment(workspace=workspace, name="nfl-injury-scraper")

In [20]:
# Publish the Pipeline
published_pipeline = pipeline.publish(name="nfl_injury_scraper_pipeline")

Created step nfl-scraper [be4382cd][a914744c-b9ff-4729-8ed0-eec397533acd], (This step will run and generate new outputs)


In [21]:
# Set the schedule recurrence (e.g., every day at 10:00 UTC)
recurrence = ScheduleRecurrence(frequency="Day", interval=1, time_of_day="12:04")

In [22]:
# Create the pipeline schedule
schedule = Schedule.create(
    workspace=workspace,
    name="nfl_scraper_schedule",
    pipeline_id=published_pipeline.id,
    experiment_name="nfl-scraper-exp",
    recurrence=recurrence,
)